In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

In [12]:
d1 = pd.read_excel('/Users/mac/Downloads/Book2.xlsx')
d2 = pd.read_excel('/Users/mac/Downloads/Book1.xlsx')


In [13]:
d1 = d1.sort_values(by='A/c').reset_index(drop=True)
d1['Sno'] = range(1,len(d1)+1)
d1.head()

,Sno,Name,OA/c,A/c,ST
0,1,GURJEEVEN SINGH S/O CHARANJIT SINGH,1003,23500103100001,119430
1,2,MANMOHAN SINGH S/O MAHINDER SINGH,992,23500103100004,65865
2,3,GURCHARAN SINGH S/O BABU SINGH,993,23500103100007,43380
3,4,PARKESH CHAND S/O HANS RAJ,990,23500103100008,-677
4,5,KARAMJIT CHAND S/O HANS RAJ,991,23500103100009,44065


In [14]:
d2.sort_values(by='A/c').reset_index(drop=True)
d2['Sno'] = range(1,len(d2)+1)

In [15]:
d_common = d1.merge(d2, on="A/c", suffixes=('_d1', '_d2'))
d_not_common = d1.merge(d2, on="A/c", how="outer", indicator=True)
# d_common['Sno_d1'] = range(1,len('Sno_d1')+1)
# d_common['Sno_d2'] = range(1,len('Sno_d2')+1)


In [16]:
only_in_d1 = d_not_common[d_not_common['_merge'] == 'left_only'].drop(columns=['_merge'])
only_in_d2 = d_not_common[d_not_common['_merge'] == 'right_only'].drop(columns=['_merge'])


In [17]:
d_common.head()

,Sno_d1,Name_d1,OA/c,A/c,ST_d1,Sno_d2,Name_d2,ST_d2
0,1,GURJEEVEN SINGH S/O CHARANJIT SINGH,1003,23500103100001,119430,1,GURJIWAN SINGH S/O CHARNJIT SING,-119430
1,2,MANMOHAN SINGH S/O MAHINDER SINGH,992,23500103100004,65865,3,MANMOHAN SINGH S/O MOHINDER SING,-65865
2,2,MANMOHAN SINGH S/O MAHINDER SINGH,992,23500103100004,65865,4,MANMOHAN SINGH S/O MOHINDER SING,-65865
3,3,GURCHARAN SINGH S/O BABU SINGH,993,23500103100007,43380,7,GURCHARN SINGH S/O BABU SINGH,-43380
4,3,GURCHARAN SINGH S/O BABU SINGH,993,23500103100007,43380,8,GURCHARN SINGH S/O BABU SINGH,-43380


In [23]:
def clean_numeric(column):
    return pd.to_numeric(
        column.astype(str)
        .str.replace(',', '', regex=False)        # Remove commas
        .str.replace(r'[^\d\.\-]', '', regex=True),  # Remove anything not digit, dot, or minus
        errors='coerce'
    ).fillna(0)

d_common['ST_d1'] = clean_numeric(d_common['ST_d1'])
d_common['ST_d2'] = clean_numeric(d_common['ST_d2'])

d_common["Difference"] = d_common["ST_d1"] + d_common["ST_d2"]
d_common = d_common[d_common['Difference']!=0]
#d_common['Sno'] = range(1,len(d1)+1)

In [24]:
d_common.head()

,Sno_d1,Name_d1,OA/c,A/c,ST_d1,Sno_d2,Name_d2,ST_d2,Difference
61,51,MAKHAN SINGH S/O LABH SINGH,1086,23500103100262,30,141,MAKHAN SINGH S/O SH LABH SINGH,-5430.0,-5400.0
121,109,SURINDER SINGH S/O GURDEV SINGH,985,23500103100524,114305,209,SURINDER SINGH S/O SH GURDEV SIN,-101055.0,13250.0
289,273,JAGJIT SINGH S/O PARTAP SINGH,11,23510010301593,-450,407,JAGJIT SINGH S/O PARTAP SINGH,0.0,-450.0
333,317,HARPAL SINGH S/O JAGMAIL SINGH,471,23510010301663,150355,464,HARPAL SINGH S/O JAGMAIL SINGH,-137430.0,12925.0
357,341,ROOP LAL S/O KARAM CHAND,4,23510010301711,-416,494,ROOP LAL S/O KARAM CHAND,0.0,-416.0


In [ ]:
d_common.to_csv("common_accounts.csv", index=False)
only_in_d1.to_excel("only_in_d1.xlsxv", index=False)
only_in_d2.to_excel("only_in_d2.xlsx", index=False)

In [87]:
d_common.to_excel("common.xlsx", index=False)

In [25]:
d_common.to_excel('/Users/mac/Downloads/common.xlsx', index=False, engine='openpyxl')
wb = load_workbook('/Users/mac/Downloads/common.xlsx')
ws = wb.active
orange_fill = PatternFill(start_color="FABF8F", end_color="FABF8F", fill_type="solid")  # Orange (Columns 1-5)
blue_fill = PatternFill(start_color="92CDDC", end_color="92CDDC", fill_type="solid")  # Blue (Columns 6-8)
green_fill = PatternFill(start_color="C4D79B", end_color="C4D79B", fill_type="solid")  # Green (Column 9)

for col_idx, column in enumerate(d_common.columns, start=1):
    cell = ws[f"{get_column_letter(col_idx)}1"]
    
    cell.font = Font(bold=True, color="000000", size=20)  # Bold + White Text + Bigger Size
    cell.alignment = Alignment(horizontal="center")  # Centered Text
    # Apply color based on column position
    if 1 <= col_idx <= 5:
        cell.fill = orange_fill
    elif 6 <= col_idx <= 8:
        cell.fill = blue_fill
    elif col_idx == 9:
        cell.fill = green_fill

# Auto Adjust Column Widths
for col_idx, column in enumerate(d_common.columns, start=1):
    max_length = max(len(str(val)) for val in d_common[column]) + 2
    ws.column_dimensions[get_column_letter(col_idx)].width = max_length

wb.save('/Users/mac/Downloads/common.xlsx')